# 07-2

## 2개의 층

In [1]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

# train_test_split() : 훈련 세트와 검증 세트로 나눔.
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

# 시그모이드 활성화 함수를 사용한 은닉층 : 100개의 뉴런을 가진 밀집층. 은닉층은 출력층보다 뉴럭 개수 많아야 함.
# activation='sigmoid' : 시그모이드 활성화 함수 사용
#  input_shape=(784,) : 입력 크기 정함.
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))

# 소프트맥스 함수를 사용한 출력층
# activation='softmax' : 소프트맥스 함수 사용.
dense2 = keras.layers.Dense(10, activation='softmax')

4423680/4422102 [==============================] - 0s 0us/step


## 심층 신경망 만들기

In [2]:
# Sequential 클래스 객체 생성시 여러 개의 층 추가하려면 dense1과 dense2를 리스트로 만들어 전달. 반드시 출력 층을 가장 마지막에 두어야 함. 가장 처음 등장하는 은닉층에서 마지막 출력 층의 순서로 나열해야 함.
model = keras.Sequential([dense1, dense2])

# 층에 대한 정보 호출
model.summary()
# 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력
# 층 생성시 name 매개변수로 이름 지정 가능, defualt 값으로 dense

# 첫번째 층
# (None, 100) = (샘플 개수, 은닉층 뉴런 개수)
# None = 아직 정의 되지 않은 상태(샘플 개수 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록)
# 100 = 100개의 출력 나옴, 샘플마다 784개의 픽셀 값이 은닉 층 통과하며 100개의 특성으로 압축됨.
# 모델 파라미터 개수 = 78500 : DENSE 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치 있음. 뉴런마다 1개의 절편 있음. 784 * 100 + 100 = 78500

# 두번째 층
# 파라미터 개수 : 100개 은닉층 뉴런과 10개의 출력층 뉴런이 모두 연결되고 출력 층 뉴런마다 하나의 절편이 있음. 100*10 + 10 = 1010

# Total params : 은닉층와 출력층 파라미터 값 합친 것 . 78500 + 1010
# Non-trainable params: 경사 하강법으로 훈련되지 않는 파라미터 층의 파라미터 개수.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


## 층을 추가하는 다른 방법 1



In [3]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


## 층을 추가하는 다른 방법 2

In [4]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))
# 한 눈에 추가되는 층을 볼 수 있고, 프로그램 실행 시 동적으로 층 선택하여 추가할 수 있음.

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [5]:
# 모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)
# 추가된 층이 성능 향상 시켰다는 것을 알 수 있음.

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5626 - accuracy: 0.8099
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4082 - accuracy: 0.8527
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3733 - accuracy: 0.8644
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3503 - accuracy: 0.8734
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3328 - accuracy: 0.8791


## 렐루 활성화 함수

In [6]:
model = keras.Sequential()
# Flatten 클래스 : 배치 차원 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할. 입력에 곱해지는 가중치나 절편이 없음. 층처럼 입력층과 은닉층 사이에 추가하기 때문에 층이라 부름. 입력층 바로 뒤에 추가.
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

# Flatten 에 포함된 모델 파라미터는 0개 -> 케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있음.

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [7]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

# 모델을 컴파일하고 훈련.
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)    

# 시그모이드 함수를 사용했을 때와 비교했을 때 성능이 향상된 것 확인 가능

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5304 - accuracy: 0.8113
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3895 - accuracy: 0.8597
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3560 - accuracy: 0.8725
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3340 - accuracy: 0.8812
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3193 - accuracy: 0.8853


In [8]:
# 검증 세트에서의 성능
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3678 - accuracy: 0.8693


[0.367831289768219, 0.8693333268165588]

## 옵티마이저

In [ ]:
# optimizer='sgd' : SGD 옵티마이저 사용.
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

# 위의 코드와 동일.
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# learning_rate=0.1 학습률 설정.
sgd = keras.optimizers.SGD(learning_rate=0.1)

In [9]:
# momentum 기본값 = 0, 0보다 큰 값으로 지정하면 이전의 그레이디언트를 사고도처럼 사용하는 모멘텀 최적화 사용(보통 0.9 이상 사용)
#  nesterov : True로 바꾸면 네스테로프 모멘텀(네스테로프 가속 경사) 최적화 사용, 네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현.
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

In [10]:
# 적응적 학습률 사용하는 대표적인 옵티마이저 : Adagrad, RMSprop
# Adgrad
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

# RMSprop
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [11]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5208 - accuracy: 0.8191
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3936 - accuracy: 0.8586
Epoch 3/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3513 - accuracy: 0.8728
Epoch 4/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3244 - accuracy: 0.8814
Epoch 5/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3077 - accuracy: 0.8870


In [12]:
# 검증세트에서의 성능
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3264 - accuracy: 0.8802


[0.3264469504356384, 0.8802499771118164]